In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [10]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [11]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [12]:
dependent=dataset[['Purchased']]

In [13]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [14]:
independent.shape

(400, 3)

In [15]:
dependent.shape

(400, 1)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [17]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
from sklearn.model_selection import GridSearchCV
param_grid={'solver': ['liblinear', 'lbfgs', 'saga', 'newton-cg'],'penalty':['l2']}

grid=GridSearchCV (LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')

grid.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['liblinear', 'lbfgs', 'saga', 'newton-cg']},
             scoring='f1_weighted', verbose=3)

In [20]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)

In [21]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [22]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [23]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro of best paramete{}:".format(grid.best_params_),f1_macro)

The f1_macro of best paramete{'penalty': 'l2', 'solver': 'liblinear'}: 0.8906190214115365


In [24]:
print("The confusion_matrix:\n",cm)

The confusion_matrix:
 [[74  5]
 [ 8 33]]


In [25]:
print ("The classification_report:\n",clf_report)

The classification_report:
               precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [26]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9484408768138315

In [27]:
table=pd.DataFrame.from_dict(re)

In [28]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008733,0.002071,0.009822,0.001252,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
1,0.011047,0.003073,0.009014,0.001001,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
2,0.009920,0.004406,0.009080,0.003857,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
3,0.009942,0.003754,0.007310,0.000599,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1


In [29]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex_male 0 or 1:"))
smoker_yes__input=int(input("Smoker_yes 0 or 1:"))

Age: 32
BMI: 43
Children: 2
Sex_male 0 or 1: 0
Smoker_yes 0 or 1: 1


In [35]:
Future_prediction=grid.predict([[age_input,bmi_input,children_input]])

In [37]:
Future_prediction

array([1], dtype=int64)

In [39]:
import pickle

In [41]:
filename="Finalaized_model_LogiticClassifier-Grid.sav"

In [43]:
pickle.dump(grid,open(filename,'wb'))

In [45]:
loaded_model=pickle.load(open("finalaized_model_LogiticClassifier-Grid.sav",'rb'))

In [47]:
result=loaded_model.predict([[243,400,1]])

In [49]:
result

array([1], dtype=int64)